## Vamos a conectar a mi MYSQL Local

#### **Si desea conectarse a MYSQL desde la nube como AWS el procedimiento es el mismo -Solo cambian las credenciales-**

##### 1.Importamos las librerias necesarias

In [1]:
# Importamos las librerias 
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import os 
import env # Archivo .py que contienen las credenciales -secretas-

##### 2. Conectamos con la base de datos

In [2]:
# Credenciales
HOST_DB = env.ROOT_BD
USER_DB = env.USER_BD
PASS_DB = env.PASS_BD
NAME_DB = env.NAME_BD

In [3]:
# Conectamos con la base de datos 

def BD_connection(host, user_db, user_pass, name_db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user_db,
            passwd=user_pass,
            database=name_db,
            port=3306,
            connection_timeout=300
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [4]:
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)

MYSQL DATABASE connection succesful


In [5]:
# verifiquemos la conexion realizando una consulta simple
if connection:
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE()")
    db = cursor.fetchone()
    print(f"Conectado a la base de datos: {db[0]}")

Conectado a la base de datos: camada51


In [6]:
# Funcion para obtener las tablas disponibles
def show_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    for table in tables:
        print(table)

In [7]:
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)
show_tables(connection)

MYSQL DATABASE connection succesful
('donantes',)
('proveedores',)
('transaccion_donantes',)
('transaccion_proveedores',)


In [8]:
# Funcion para describir la estructura de una tabla
def describe_table(connection, table_name):
    cursor = connection.cursor()
    query = f"DESCRIBE {table_name}"
    cursor.execute(query)
    columns = cursor.fetchall()
    for column in columns:
        print(column)

In [9]:
describe_table(connection, 'proveedores')

('id_proveedor', 'int', 'NO', 'PRI', None, 'auto_increment')
('nombre', 'varchar(50)', 'NO', '', None, '')
('cuit', 'varchar(50)', 'NO', '', None, '')
('tipo_contribuyente', 'varchar(50)', 'NO', '', None, '')
('correo_electronico', 'varchar(50)', 'NO', '', None, '')
('nro_cuenta', 'varchar(50)', 'NO', '', None, '')
('razon_social', 'varchar(50)', 'NO', '', None, '')
('telefono', 'varchar(50)', 'NO', '', None, '')
('nombre_ciudad', 'varchar(100)', 'YES', '', None, '')


In [10]:
describe_table(connection, 'transaccion_proveedores')

('id_transaccion', 'int', 'NO', 'PRI', None, 'auto_increment')
('id_proveedor', 'int', 'NO', 'MUL', None, '')
('importe', 'decimal(10,2)', 'NO', '', None, '')
('fecha', 'date', 'YES', '', None, '')


##### No tenemos datos ingresados en nuestra base de datos, vamos a gregarle datos

##### 3. Agregamos datos a nuestras 3 tablas

Creamos el motor de SQL_ALCHEMY para poder ingresar, consultar y extrar datos desde la Base de Datos

In [11]:
# Crear el engine de SQLAlchemy usando la conexion MySQL
def create_sqlalchemy_engine(host_name, user_bd, password_bd, name_bd):
    url = f"mysql+mysqlconnector://{user_bd}:{password_bd}@{host_name}/{name_bd}"
    return create_engine(url)

In [12]:
# Crear el engine
engine = create_sqlalchemy_engine(HOST_DB, USER_DB, PASS_DB, NAME_DB)

In [13]:
# Traemos los datos
data_prove = 'Data/proveedor.csv'
data_tran = 'Data/transaccion.csv'
data_ubica = 'Data/ubicacion.csv'

In [14]:
# Los convertimos a Dataframe
proveedor = pd.read_csv(data_prove)
transaccion = pd.read_csv(data_tran)
ubicacion = pd.read_csv(data_ubica)

In [15]:
# Creamos un punto de reinicio para los datos
df_proveedor = pd.DataFrame(proveedor)
df_transaccion = pd.DataFrame(transaccion)
df_ubicacion = pd.DataFrame(ubicacion)

In [16]:
# Verificamos si tenemos los datos
df_proveedor.head(4)

,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,telefono,nro_cuenta,razon_social,id_ubicion
0,1,Servicios Alpha S.A.,30192837536,RESPONSABLE INSCRIPTO,servicios.alpha@correo.com,+54 987531525,516000,S.A,10
1,2,Proveedora Consultora,30192837558,RESPONSABLE INSCRIPTO,proveedora.consultora@empresa.com,+54 987531529,514100,S.A,12
2,3,Pedro Morales,30975312552,RESPONSABLE INSCRIPTO,pedro.morales@correo.com,+54 987531536,514100,S.A,24
3,4,Compañía Zeta,30135792477,MONOTRIBUTISTA,gabriela.torres@empresa.com,+54 987531525,514100,S.A,1


In [17]:
# Cambiamos la columna id_ubicion a id_ubicacion
df_proveedor.rename(columns={'id_ubicion': 'id_ubicacion'}, inplace=True)
df_proveedor.head(1)

,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,telefono,nro_cuenta,razon_social,id_ubicacion
0,1,Servicios Alpha S.A.,30192837536,RESPONSABLE INSCRIPTO,servicios.alpha@correo.com,+54 987531525,516000,S.A,10


In [18]:
# Ordenamos las columnas
df_proveedor = df_proveedor[['id_proveedor', 'nombre', 'cuit', 'tipo_contribuyente', 'correo_electronico', 'nro_cuenta', 'razon_social', 'telefono', 'id_ubicacion']]
df_proveedor.head(1)


,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,nro_cuenta,razon_social,telefono,id_ubicacion
0,1,Servicios Alpha S.A.,30192837536,RESPONSABLE INSCRIPTO,servicios.alpha@correo.com,516000,S.A,+54 987531525,10


In [19]:
# Verificamos duplicados
print(df_proveedor.duplicated().sum())
# Si hay 8 duplicados, ahora eliminamos los duplicados
df_proveedor = df_proveedor.drop_duplicates()

8


In [20]:
# Verificamos cuantos duplicados hay en la columna id
print(df_proveedor['id_proveedor'].duplicated().sum())

381


In [21]:
# Buscamos en que filas hay valores duplicados
duplicados = df_proveedor[df_proveedor['id_proveedor'].duplicated()]
duplicados.sample(6)


,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,nro_cuenta,razon_social,telefono,id_ubicacion
412,117,Diego Fernández,30-12345678-5,RESPONSABLE INSCRIPTO,diego fernández@empresa.com,509100,S.R.L,+5689632149,20
171,86,Grupo Alfa,30-97531246-1,RESPONSABLE INSCRIPTO,grupo alfa@empresa.com,501400,S.R.L,+5689632144,32
31,23,Rosa Pérez,30-24681358-6,RESPONSABLE INSCRIPTO,rosa pérez@empresa.com,509100,S.A,+5689632144,18
207,93,Isabel Ramírez,33-76543218-9,RESPONSABLE INSCRIPTO,isabel ramírez@empresa.com,516000,S.A,+5689632149,26
82,44,Carlos Méndez,30-12345678-9,MONOTRIBUTISTA,carlos méndez@empresa.com,516000,S.A,+5689632149,39
349,95,Sofía Martínez,27-12345678-9,Responsable Inscripto,sofía martínez@empresa.com,501400,S.A,+5689632149,15


In [22]:
# vamos a revisar si hay valores nulos
df_proveedor.isna().sum()

id_proveedor          0
nombre                0
cuit                  5
tipo_contribuyente    0
correo_electronico    0
nro_cuenta            0
razon_social          0
telefono              0
id_ubicacion          0
dtype: int64

In [23]:
# LLenamos los valores faltantes con 0
df_proveedor['cuit'] = df_proveedor['cuit'].fillna('0')
df_proveedor.head()

,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,nro_cuenta,razon_social,telefono,id_ubicacion
0,1,Servicios Alpha S.A.,30192837536,RESPONSABLE INSCRIPTO,servicios.alpha@correo.com,516000,S.A,+54 987531525,10
1,2,Proveedora Consultora,30192837558,RESPONSABLE INSCRIPTO,proveedora.consultora@empresa.com,514100,S.A,+54 987531529,12
2,3,Pedro Morales,30975312552,RESPONSABLE INSCRIPTO,pedro.morales@correo.com,514100,S.A,+54 987531536,24
3,4,Compañía Zeta,30135792477,MONOTRIBUTISTA,gabriela.torres@empresa.com,514100,S.A,+54 987531525,1
4,5,Servicios Consultores,30975312566,RESPONSABLE INSCRIPTO,servicios.consultores@correo.com,506100,S.A,+54 987531529,26


In [24]:
# Encontramos que si tenemos valores duplicados, pero lo unico que lo diferencia es el id_ubicacion eso significa que para cada proveedor puede haber diferentes ubicaciones
df_unificado = pd.merge(
    df_proveedor,           # DataFrame de proveedores
    df_ubicacion,           # DataFrame de ciudades
    left_on='id_ubicacion', # Columna en df_proveedor
    right_on='id_ciudad',   # Columna en df_ubicaciones
    how='left'              # Mantener todos los proveedores aunque no tengan ciudad (ajusta según necesidad)
)

# Eliminar columnas redundantes
df_unificado = df_unificado.drop(columns=['id_ubicacion', 'id_ciudad'])

df_unificado.head()

,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,nro_cuenta,razon_social,telefono,nombre_ciudad
0,1,Servicios Alpha S.A.,30192837536,RESPONSABLE INSCRIPTO,servicios.alpha@correo.com,516000,S.A,+54 987531525,Santiago del Estero
1,2,Proveedora Consultora,30192837558,RESPONSABLE INSCRIPTO,proveedora.consultora@empresa.com,514100,S.A,+54 987531529,Luján
2,3,Pedro Morales,30975312552,RESPONSABLE INSCRIPTO,pedro.morales@correo.com,514100,S.A,+54 987531536,Río Cuarto
3,4,Compañía Zeta,30135792477,MONOTRIBUTISTA,gabriela.torres@empresa.com,514100,S.A,+54 987531525,Villa Allende
4,5,Servicios Consultores,30975312566,RESPONSABLE INSCRIPTO,servicios.consultores@correo.com,506100,S.A,+54 987531529,Salta


In [25]:
# Verificar si 'RESPONSABLE INSCRIPTO' aparece (ignorando mayúsculas/minúsculas)
mask_responsable = df_proveedor['tipo_contribuyente'].str.contains('responsable inscripto', case=False)

# Mostrar los resultados
df_proveedor[mask_responsable]

,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,nro_cuenta,razon_social,telefono,id_ubicacion
0,1,Servicios Alpha S.A.,30192837536,RESPONSABLE INSCRIPTO,servicios.alpha@correo.com,516000,S.A,+54 987531525,10
1,2,Proveedora Consultora,30192837558,RESPONSABLE INSCRIPTO,proveedora.consultora@empresa.com,514100,S.A,+54 987531529,12
2,3,Pedro Morales,30975312552,RESPONSABLE INSCRIPTO,pedro.morales@correo.com,514100,S.A,+54 987531536,24
4,5,Servicios Consultores,30975312566,RESPONSABLE INSCRIPTO,servicios.consultores@correo.com,506100,S.A,+54 987531529,26
5,6,Servicios Proveedores,30135792534,RESPONSABLE INSCRIPTO,servicios.proveedores@correo.com,501400,S.A,+54 987531536,3
...,...,...,...,...,...,...,...,...,...
519,100,Roberto Cruz,27-23456789-3,Responsable Inscripto,roberto cruz@empresa.com,501400,S.A,+5689632149,14
520,104,Silvia Ramírez,30-76543214-8,RESPONSABLE INSCRIPTO,silvia ramírez@empresa.com,509100,S.R.L,+5689632144,2
522,27,Gabriela Torres,33-76543219-7,RESPONSABLE INSCRIPTO,gabriela torres@empresa.com,516000,S.R.L,+5689632149,36
524,111,María González,23-45678901-3,Responsable Inscripto,maría gonzález@empresa.com,502100,S.R.L,+5689632149,2


In [26]:
# Cambiamos esto a una expresion regular 
# Eliminar espacios en los nombres de las columnas
df_unificado.columns = df_unificado.columns.str.strip()

# Reemplazar "RESPONSABLE INSCRIPTO" por "Responsable inscrito" en la columna correcta
df_unificado['tipo_contribuyente'] = df_unificado['tipo_contribuyente'].str.replace(r'(?i)responsable inscripto', 'Responsable inscrito', regex=True)

# Ahora con MONOTRIBUTISTA
df_unificado['tipo_contribuyente'] = df_unificado['tipo_contribuyente'].str.replace(r'(?i)Monotributista', 'Monotributista', regex=True)

# Mostrar el DataFrame con los cambios
df_unificado.head(4)

,id_proveedor,nombre,cuit,tipo_contribuyente,correo_electronico,nro_cuenta,razon_social,telefono,nombre_ciudad
0,1,Servicios Alpha S.A.,30192837536,Responsable inscrito,servicios.alpha@correo.com,516000,S.A,+54 987531525,Santiago del Estero
1,2,Proveedora Consultora,30192837558,Responsable inscrito,proveedora.consultora@empresa.com,514100,S.A,+54 987531529,Luján
2,3,Pedro Morales,30975312552,Responsable inscrito,pedro.morales@correo.com,514100,S.A,+54 987531536,Río Cuarto
3,4,Compañía Zeta,30135792477,Monotributista,gabriela.torres@empresa.com,514100,S.A,+54 987531525,Villa Allende


In [27]:
# Eliminaremos los duplicados y nos quedaron solo con un proveedor
df_unificado = df_unificado.drop_duplicates(subset=['id_proveedor'], keep='first')

In [28]:
df_transaccion.head(4)

,id_proveedor,importe,fecha
0,1,215609,2024-06-30
1,2,124891,2024-06-30
2,3,209036,2024-06-29
3,4,121389,2024-06-29


In [28]:
# Paso 1: Crear la columna 'id_transaccion' (puede ser un número único por ejemplo)
df_transaccion['id_transaccion'] = range(1, len(df_transaccion) + 1)

# Paso 2: Reordenar las columnas para que 'id_transaccion' esté primero
df_transaccion = df_transaccion[['id_transaccion', 'id_proveedor', 'importe', 'fecha']]

# Mostrar el dataframe final
print(df_transaccion.head(1))

   id_transaccion  id_proveedor  importe       fecha
0               1             1   215609  2024-06-30


In [29]:
# Insertamos los datos proveedor
df_unificado.to_sql(name= 'proveedores', con = engine, if_exists='append', index=False)

-1

In [30]:
# transaccion
df_transaccion.to_sql(name= 'transaccion_proveedores', con = engine, if_exists='append', index=False)

-1

##### 4. Realizamos algunas consultas 

In [31]:
# Query para conocer la suma de los importes de los proveedores
query_importe = "SELECT id_proveedor, SUM(importe) AS Total_importe FROM transaccion_proveedores GROUP BY id_proveedor;"
query_sumtotal= "SELECT SUM(importe) FROM transaccion_proveedores"
query_count   = "SELECT COUNT(*) FROM transaccion_proveedores"

In [32]:
# comprabamos la respuesta
importe = pd.read_sql(query_importe, engine)
print(importe)

     id_proveedor  Total_importe
0               1       431218.0
1               2       249782.0
2               3       418072.0
3               4       364167.0
4               5       161682.0
..            ...            ...
134           135      1802488.0
135           136      1876343.0
136           137      1481034.0
137           138      1758696.0
138           139      1524600.0

[139 rows x 2 columns]


In [33]:
# comprabamos la respuesta
suma_importe = pd.read_sql(query_sumtotal, engine)
print(suma_importe)

   SUM(importe)
0    78116931.0


In [34]:
# comprabamos la respuesta
count = pd.read_sql(query_count, engine)
print(count)

   COUNT(*)
0       528


##### 5. Agregamos más datos

##### Listo conexion exitosa, ahora voy agregar mi archivo excel, para agregar donantes

In [50]:
# Traemos solo la hoja donantes del archivo excel

data = pd.read_excel('Data/Donantes Sprint 2.xlsx')

In [51]:
# convertimos en dataframe, para tener un punto de reinicio
df_donantes = pd.DataFrame(data)
df_donantes.head(4)

,Número,Nombre,Tipo,Contacto,Cargo,Correo Electrónico,Paises,Teléfono,Observaciones,Razon Social,...,CUIT,Alta,Baja,Estado de donantes,Frecuencia,Fecha_Donación,Mes,Año,Importe,Nro de Cuenta
0,D00001,Fundación Empresarial,Empresa,Contacto VVVV,-,Fundación Empresarial@example.com,Argentina,(011) 5000-0105,NaN,SA,...,29-01234576-14,2024-06-04,NaT,Activo,Anual,2022-10-04,oct,2022,150000,409021
1,D00002,Apoyo Corporativo S.A.,Empresa,Contacto VVVV,-,Apoyo Corporativo S.A.@example.com,Argentina,(011) 5000-0104,Observación 104,SA,...,29-01234576-13,2024-06-03,NaT,Activo,Anual,2022-12-04,dic,2022,120000,409021
2,D00003,Donativos y Más S.R.L.,Empresa,Contacto VVVV,-,Donativos y Más S.R.L.@example.com,Argentina,(011) 5000-0103,Observación 103,SRL,...,29-01234576-12,2024-06-02,NaT,Activo,Mensual,2023-02-04,feb,2023,156325,402102
3,D00004,Ayuda Empresarial S.A.,Empresa,Contacto VVVV,-,Ayuda Empresarial S.A.@example.com,Argentina,(011) 5000-0102,Observación 102,SA,...,29-01234576-11,2024-06-01,NaT,Activo,Anual,2023-04-04,abr,2023,162961,403101


In [52]:
# Filtramos las filas para ingresarlas en la tabla transaccion_donantes
# Seleccionar las columnas relacionadas con las transacciones
df_transacciones_donantes = df_donantes[['Número', 'Fecha_Donación', 'Mes', 'Año', 'Importe', 'Nro de Cuenta']]

# Agregar un identificador único para las transacciones (opcional)
df_transacciones_donantes['id_transaccion'] = range(1, len(df_transacciones_donantes) + 1)

# Reordenamos las columnas para mejor compresion
df_transacciones_donantes = df_transacciones_donantes[['id_transaccion', 'Número', 'Fecha_Donación', 'Mes', 'Año', 'Importe', 'Nro de Cuenta']]

# verificamos 
df_transacciones_donantes.head(4)


C:\Users\DELL\AppData\Local\Temp\ipykernel_13884\1393800676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transacciones_donantes['id_transaccion'] = range(1, len(df_transacciones_donantes) + 1)


,id_transaccion,Número,Fecha_Donación,Mes,Año,Importe,Nro de Cuenta
0,1,D00001,2022-10-04,oct,2022,150000,409021
1,2,D00002,2022-12-04,dic,2022,120000,409021
2,3,D00003,2023-02-04,feb,2023,156325,402102
3,4,D00004,2023-04-04,abr,2023,162961,403101


In [53]:
# Verificamos si hay duplicados o faltantes
df_transacciones_donantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106 entries, 0 to 1105
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_transaccion  1106 non-null   int64         
 1   Número          1106 non-null   object        
 2   Fecha_Donación  1106 non-null   datetime64[ns]
 3   Mes             1106 non-null   object        
 4   Año             1106 non-null   int64         
 5   Importe         1106 non-null   int64         
 6   Nro de Cuenta   1106 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 60.6+ KB


In [54]:
# Modificamos el df original
donantes = df_donantes.drop_duplicates(subset=['Número']).copy()

# ELiminamos las columnas que se encuentran en transacciones_donantes 
donantes = donantes.drop(columns=['Fecha_Donación', 'Mes', 'Año', 'Importe', 'Nro de Cuenta'], errors='ignore')

# Mostrar una muestra de los donantes únicos
donantes.head(4)

,Número,Nombre,Tipo,Contacto,Cargo,Correo Electrónico,Paises,Teléfono,Observaciones,Razon Social,Tipo de Contribuyente,CUIT,Alta,Baja,Estado de donantes,Frecuencia
0,D00001,Fundación Empresarial,Empresa,Contacto VVVV,-,Fundación Empresarial@example.com,Argentina,(011) 5000-0105,NaN,SA,Responsable Inscripto,29-01234576-14,2024-06-04,NaT,Activo,Anual
1,D00002,Apoyo Corporativo S.A.,Empresa,Contacto VVVV,-,Apoyo Corporativo S.A.@example.com,Argentina,(011) 5000-0104,Observación 104,SA,Responsable Inscripto,29-01234576-13,2024-06-03,NaT,Activo,Anual
2,D00003,Donativos y Más S.R.L.,Empresa,Contacto VVVV,-,Donativos y Más S.R.L.@example.com,Argentina,(011) 5000-0103,Observación 103,SRL,Responsable Inscripto,29-01234576-12,2024-06-02,NaT,Activo,Mensual
3,D00004,Ayuda Empresarial S.A.,Empresa,Contacto VVVV,-,Ayuda Empresarial S.A.@example.com,Argentina,(011) 5000-0102,Observación 102,SA,Responsable Inscripto,29-01234576-11,2024-06-01,NaT,Activo,Anual


In [56]:
# Limpiamos los valores faltantes
donantes['Baja'] = donantes['Baja'].fillna('-')

# Reemplazamos valores nulos en las demas filas
columns_to_fill = ['Contacto', 'Observaciones', 'Razon Social']
donantes[columns_to_fill] = donantes[columns_to_fill].fillna('-')

# Verificar que ya no hay valores nulos
print(donantes.isnull().sum())

Número                   0
Nombre                   0
Tipo                     0
Contacto                 0
Cargo                    0
Correo Electrónico       0
Paises                   0
Teléfono                 0
Observaciones            0
Razon Social             0
Tipo de Contribuyente    0
CUIT                     0
Alta                     0
Baja                     0
Estado de donantes       0
Frecuencia               0
dtype: int64


In [57]:
# Revisamos los df antes de insertar los datos 
donantes.head(4)

,Número,Nombre,Tipo,Contacto,Cargo,Correo Electrónico,Paises,Teléfono,Observaciones,Razon Social,Tipo de Contribuyente,CUIT,Alta,Baja,Estado de donantes,Frecuencia
0,D00001,Fundación Empresarial,Empresa,Contacto VVVV,-,Fundación Empresarial@example.com,Argentina,(011) 5000-0105,-,SA,Responsable Inscripto,29-01234576-14,2024-06-04,-,Activo,Anual
1,D00002,Apoyo Corporativo S.A.,Empresa,Contacto VVVV,-,Apoyo Corporativo S.A.@example.com,Argentina,(011) 5000-0104,Observación 104,SA,Responsable Inscripto,29-01234576-13,2024-06-03,-,Activo,Anual
2,D00003,Donativos y Más S.R.L.,Empresa,Contacto VVVV,-,Donativos y Más S.R.L.@example.com,Argentina,(011) 5000-0103,Observación 103,SRL,Responsable Inscripto,29-01234576-12,2024-06-02,-,Activo,Mensual
3,D00004,Ayuda Empresarial S.A.,Empresa,Contacto VVVV,-,Ayuda Empresarial S.A.@example.com,Argentina,(011) 5000-0102,Observación 102,SA,Responsable Inscripto,29-01234576-11,2024-06-01,-,Activo,Anual


In [59]:
# Cambiamos los nombres de la columna Número a sin tilde
donantes = donantes.rename(columns={"Número" : "Numero"}) 

df_transacciones_donantes = df_transacciones_donantes.rename(columns={"Número" : "Numero"}) 

##### Corrigiendo errores comunes

In [60]:
# Convertir la columna 'Baja' a tipo datetime, convirtiendo los valores '-' a None (NULL en MySQL)
donantes['Baja'] = pd.to_datetime(donantes['Baja'], errors='coerce')
print(donantes['Baja'].isna().sum())

donantes['Baja'] = donantes['Baja'].where(donantes['Baja'].notna(), None)
print(donantes['Baja'].isna().sum())  

90
90


C:\Users\DELL\AppData\Local\Temp\ipykernel_13884\2727561787.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  donantes['Baja'] = pd.to_datetime(donantes['Baja'], errors='coerce')


In [61]:
# Cambiamos los nombres de la columnas
donantes = donantes.rename(columns={"Correo Electrónico" : "Correo_Electronico"})
donantes = donantes.rename(columns={"Teléfono" : "Telefono"})
donantes = donantes.rename(columns={"Razon Social" : "Razon_Social"}) 
donantes = donantes.rename(columns={"Tipo de Contribuyente" : "Tipo_de_Contribuyente"})
donantes = donantes.rename(columns={"Estado de donantes" : "Estado_de_donantes"})
 

In [62]:
# Cambiamos los nombre el df_donantes_transaccion
df_transacciones_donantes = df_transacciones_donantes.rename(columns={"Fecha_Donación" : "Fecha_Donacion"})
df_transacciones_donantes = df_transacciones_donantes.rename(columns={"Nro de Cuenta" : "Nro_de_Cuenta"})


In [48]:
# Insertamos los datos de los donantes
donantes.to_sql(name= 'donantes', con = engine, if_exists='append', index=False)

-1

In [63]:
# Insertamos los datos de las transacciones de los donantes
df_transacciones_donantes.to_sql(name= 'transaccion_donantes', con = engine, if_exists='append', index=False)


-1

#### Listo !!! Terminamos de ingresar los datos al MYSQL con Python 